# Ethereum 2.0 Merge Proof-of-Concept

This notebook provides a proof-of-concept for the Ethereum 2.0 Quick Merge Proposal ([Official Spec](https://github.com/ethereum/eth2.0-specs/tree/dev/specs/merge), [Vitalik's initial post](https://notes.ethereum.org/@vbuterin/B1mUf6DXO)).

The following Ethereum specs have been used:
- [`py-evm`](https://github.com/ethereum/py-evm)
- [Eth2 Merge Spec](https://github.com/ethereum/eth2.0-specs/tree/dev/specs/merge)

## Prerequisites
- Python3, Jupyter Notebook
- Run `make install` to:
    - create a venv.
    - download and install Eth1 & Eth2 python modules in the venv.
    - install this repo as a local package in the venv.
    - install the venv as a kernel (named `merge-spec-poc`) for Jupyter notebook.
- Choose the `merge-spec-poc` kernel to execute this Jupyter notebook.

In [1]:
from eth1 import Eth1Rpc
from eth.chains.base import MiningChain, Chain
from eth.vm.forks import IstanbulVM
import eth.tools.builder.chain as builder
        
# Initialize Eth1 chain builder
Eth1Chain = builder.build(MiningChain, builder.fork_at(IstanbulVM, 0))
Eth1Chain = builder.enable_pow_mining(Eth1Chain)
eth1_rpc = Eth1Rpc(Eth1Chain)

# Build a short Eth1 chain
eth1_blocks = [eth1_rpc.consensus_assembleBlock() for i in range(9)]
eth1_rpc.print_block(eth1_blocks[-1])

Eth1 Block #9
	Parent Hash:	0x9e6ba8ba92933407dc9bf4a559de2033f3b314817bd409b5743596560681ef6c
	Block Hash:	0x373cbe207d0c31e865a4306d672bd375defaf469a02435ad775437572530f99c
	Block Score:	10


In [2]:
import eth2spec.merge.spec as spec

# Set TRANSITION_TOTAL_DIFFICULTY
spec.TRANSITION_TOTAL_DIFFICULTY = 10

# Define verify_execution_state_transition
def _verify_execution_state_transition(execution_payload: spec.ExecutionPayload) -> bool:
    # Assumption: Eth1 p2p has gossiped the block already
    return eth1_rpc.is_accepted_block(execution_payload.block_hash)

def _get_pow_block(block_hash: spec.Hash32) -> spec.PowBlock:
    pow_block = spec.PowBlock(
        block_hash=block_hash,
        is_processed=False,
        is_valid=False,
        total_difficulty=0,
    )
    if eth1_rpc.is_accepted_block(block_hash):
        block = eth1_rpc.get_block_by_hash(block_hash)
        pow_block.is_processed = True
        pow_block.is_valid = True
        pow_block.total_difficulty = eth1_rpc.get_score(block_hash)
    return pow_block

spec.verify_execution_state_transition = _verify_execution_state_transition
spec.get_pow_block = _get_pow_block

In [3]:
# Eth2 chain building tools
from eth2spec.test.helpers import genesis

genesis_state = genesis.create_genesis_state(spec, [32*10**9 for i in range(100)], 16*10**9)
genesis_block = spec.BeaconBlock(state_root=genesis_state.hash_tree_root())
store = spec.get_forkchoice_store(genesis_state, genesis_block)

In [4]:
from eth2spec.test.helpers.block import build_empty_block_for_next_slot, sign_block
from eth2spec.test.helpers.state import state_transition_and_sign_block, next_epoch

# Push the Eth2 state to some arbitraty slot
time = spec.SECONDS_PER_SLOT * 1000
spec.on_tick(store, time)
state = genesis_state.copy()
next_epoch(spec, state)

# Build a short Eth2 chain
for i in range(5):
    block = build_empty_block_for_next_slot(spec, state)
    block.body.execution_payload = spec.ExecutionPayload()
    signed_block = state_transition_and_sign_block(spec, state, block)

def print_eth2_state(state):
    print("Eth2 State:")
    print(f"\tSlot: {state.slot}")
    print(f"\tPrev. Eth1 Block: {state.latest_execution_payload_header.parent_hash}")
    if spec.is_transition_completed(state):
        eth1_rpc.print_block(eth1_rpc.chain.get_block_by_hash(state.latest_execution_payload_header.block_hash), "\t")

# This is the Eth2 state before the merge
print_eth2_state(state)

Eth2 State:
	Slot: 37
	Prev. Eth1 Block: 0x0000000000000000000000000000000000000000000000000000000000000000


In [5]:
def package_execution_payload(eth1_block):
    return spec.ExecutionPayload(
            block_hash=eth1_block.hash,
            parent_hash=eth1_block.header.parent_hash,
            coinbase=eth1_block.header.coinbase,
            state_root=eth1_block.header.state_root,
            number=eth1_block.header.block_number,
            gas_limit=eth1_block.header.gas_limit,
            gas_used=eth1_block.header.gas_used,
            timestamp=eth1_block.header.timestamp,
            receipt_root=eth1_block.header.receipt_root,
            # logs_bloom: ByteVector[BYTES_PER_LOGS_BLOOM]
            # transactions: List[OpaqueTransaction, MAX_APPLICATION_TRANSACTIONS]
        )

# TRANSITION_TOTAL_DIFFICULTY is set to 10
# eth1_blocks[7] is the last PoW block
# eth1_blocks[8] is the first transition block

block = build_empty_block_for_next_slot(spec, state)
# Fill in the last PoW mined block
block.body.execution_payload = package_execution_payload(eth1_blocks[8])
block.body.execution_payload.timestamp = spec.compute_time_at_slot(state, state.slot+1)
signed_block = state_transition_and_sign_block(spec, state, block)
print_eth2_state(state)

Eth2 State:
	Slot: 38
	Prev. Eth1 Block: 0x9e6ba8ba92933407dc9bf4a559de2033f3b314817bd409b5743596560681ef6c
	Eth1 Block #9
		Parent Hash:	0x9e6ba8ba92933407dc9bf4a559de2033f3b314817bd409b5743596560681ef6c
		Block Hash:	0x373cbe207d0c31e865a4306d672bd375defaf469a02435ad775437572530f99c
		Block Score:	10


## The Merge has happened!
## The last PoW block has been included in the Eth2 chain.
## All future Eth1 blocks will be produced by Eth2 PoS validator.

In [6]:
# Progress the Eth2 chain after the merge
for i in range(5):
    block = build_empty_block_for_next_slot(spec, state)
    # The eth1_rpc.mine_block() call in the next line is run by the Eth2 validator on its Eth1 node.
    # These blocks are not PoW intensive -- their difficulty will be 1.
    eth1_block = eth1_rpc.consensus_assembleBlock()
    block.body.execution_payload = package_execution_payload(eth1_block)
    block.body.execution_payload.timestamp = spec.compute_time_at_slot(state, state.slot+1)
    signed_block = state_transition_and_sign_block(spec, state, block)
    print_eth2_state(state)

Eth2 State:
	Slot: 39
	Prev. Eth1 Block: 0x373cbe207d0c31e865a4306d672bd375defaf469a02435ad775437572530f99c
	Eth1 Block #10
		Parent Hash:	0x373cbe207d0c31e865a4306d672bd375defaf469a02435ad775437572530f99c
		Block Hash:	0x2bb2d2c1188d68ece91b4793516d68bb367de0cad33406bdf57e31985213a670
		Block Score:	11
Eth2 State:
	Slot: 40
	Prev. Eth1 Block: 0x2bb2d2c1188d68ece91b4793516d68bb367de0cad33406bdf57e31985213a670
	Eth1 Block #11
		Parent Hash:	0x2bb2d2c1188d68ece91b4793516d68bb367de0cad33406bdf57e31985213a670
		Block Hash:	0x3fe32a953b623ea50eccd4bce329e9994df54ea04cb212ddc2e3739c2fae9b87
		Block Score:	12
Eth2 State:
	Slot: 41
	Prev. Eth1 Block: 0x3fe32a953b623ea50eccd4bce329e9994df54ea04cb212ddc2e3739c2fae9b87
	Eth1 Block #12
		Parent Hash:	0x3fe32a953b623ea50eccd4bce329e9994df54ea04cb212ddc2e3739c2fae9b87
		Block Hash:	0x30cc3c2118862348077f1eceaf0c0c9489421bdcffc9de411aa0c5e9a4b83d54
		Block Score:	13
Eth2 State:
	Slot: 42
	Prev. Eth1 Block: 0x30cc3c2118862348077f1eceaf0c0c9489421bdcff